# Statistical Methods Project

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import shapiro
from scipy.stats.mstats import winsorize
import statsmodels.formula.api as smf
import statsmodels.api as sm
import scipy.stats as stats
from statsmodels.stats.stattools import durbin_watson
from statsmodels.stats.diagnostic import het_white

# EDA (data description, data preprocessing)

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/ov3ipo/SM_Project/main/life_expectancy.csv")
# remove trailing space in columns name and format display function
pd.options.display.float_format = '{:.4f}'.format
df = df.rename(columns=lambda x: x.strip())

# overview on data statistic
display(df.head(10))
display(df.info())

# get quantitative and qualitative data
numeric_cols = df.drop(columns=["Status", "Country"], axis=1).columns

KeyboardInterrupt: 

## Data description

### Univariate

#### Qualitative

In [ ]:
plt.figure(figsize = (10,5))
plt.subplot(1, 2, 1)
x = df['Status'].value_counts().reset_index()
plt.pie(x=x['count'], labels=x['Status'], autopct="%0.1f%%")
plt.subplot(1, 2, 2)
sns.countplot(df, x="Status")
plt.tight_layout()
plt.show()

#### Quantitative

In [ ]:
plt.figure(figsize=(10, 20))
for i, col in enumerate(numeric_cols):
    plt.subplot(10, 2, i + 1)
    sns.histplot(df, x=col, bins=30, kde=True)
plt.tight_layout()
plt.show()

### Bivariate

In [ ]:
plt.figure(figsize=(10, 20))
index = np.argwhere(numeric_cols=="Life expectancy")
for i, col in enumerate(np.delete(numeric_cols, index)):
    plt.subplot(10, 2, i + 1)
    sns.scatterplot(df, x=col, y="Life expectancy", hue="Status", legend="auto")
plt.tight_layout()
plt.show()

### Overall statistic

In [ ]:
display(df.describe().T)

## Data preprocessing (NAs, outliers, duplicateds, label encoding)

### Missing

In [ ]:
print("\nPreprocessing\n")
print(df.isna().sum())
df = df.interpolate(method='linear', limit_direction='forward')
print("\nPostprocessing\n")
print(df.isna().sum())

### Duplicated

In [ ]:
print(f"Total duplicated values: {df.duplicated().sum()}")

### Outliers

#### Detect outliers

In [ ]:
plt.figure(figsize=(10, 15))
for i, col in enumerate(numeric_cols):
    plt.subplot(10, 2, i + 1)
    sns.boxplot(df, x=col)
plt.tight_layout()
plt.show()

# detect outliers
def detectOutliers(data):
    outliers_arr = []
    for col in data.columns:
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = data[(data[col] < lower_bound) | (data[col] > upper_bound)][col].count()
        outliers_arr.append(outliers)
    return pd.DataFrame(outliers_arr, index=data.columns, columns=["Total outliers"])

numeric_data = df.drop(columns=["Status", "Country"], axis=1)
outliers = detectOutliers(numeric_data)
display(outliers)

Why we should not use variable with high outliers -> because regression can heavily be affected by these outliers, hence we should only choose those with low outliers
Potential variable for regression of target
- Year
- Adult Mortality
- Alcohol
- BMI
- Total expenditure
- thinness 1-19 years
- thinness 5-9 years
- Income composition of resources
- Schooling

One reason is that there was a large differences between developed, devloping and underdeveloped country.
This can be the primary factor explained why outliers existed. Hence they are not abnormal observations.
->Assumption 1: OUTLIERS SHOULD NOT BE REMOVED

#### Dealing with outliers

In [ ]:
potential_var = outliers[(outliers["Total outliers"] < 150)].index
outliers_var = outliers[(outliers["Total outliers"] > 0) & (outliers["Total outliers"] < 150)].index
df_outliers = df.copy()

##### With ouliers

In [ ]:
display(detectOutliers(df_outliers[potential_var]))
plt.figure(figsize=(10, 15))
for i, col in enumerate(potential_var):
    plt.subplot(10, 2, i + 1)
    sns.boxplot(df_outliers, x=col)
plt.tight_layout()
plt.show()

##### Without ouliers

In [ ]:
for col in outliers_var:
    if col != "BMI":
        df_outliers[col] = winsorize(df_outliers[col], limits=[0.05, 0.05])

display(detectOutliers(df_outliers[potential_var]))
plt.figure(figsize=(10, 15))
for i, col in enumerate(potential_var):
    plt.subplot(10, 2, i + 1)
    sns.boxplot(df_outliers, x=col)
plt.tight_layout()
plt.show()

### Comprare statistic between with and without outliers

In [ ]:
# assign new dataframe to use for regression
df_reg_n = df[potential_var]
df_reg_y = df_outliers[potential_var]
display(df_reg_n.describe().T)
display(df_reg_y.describe().T)

# Linear Regression Analysis

## Correlation Matrix

### With outliers

In [ ]:
numeric_df = df.drop(columns=["Country", "Status"])
plt.figure(figsize=(15, 10))
sns.heatmap(numeric_df.corr(), annot=True)
plt.title('Heatmap Correlation')
plt.show()

# get variables that has high correlation with Life expectancy
corrs = numeric_df.corr()['Life expectancy'].drop('Life expectancy')
high_corr = corrs[corrs.abs() > 0.5]
print("Variables have correlation larger than 0.5: ")
print(high_corr)

### Without outliers

In [ ]:
numeric_df_outliers = df_outliers.drop(columns=["Country", "Status"])
plt.figure(figsize=(15, 10))
sns.heatmap(numeric_df_outliers.corr(), annot=True)
plt.title('Heatmap Correlation')
plt.show()

# get variables that has high correlation with Life expectancy
corrs = numeric_df_outliers.corr()['Life expectancy'].drop('Life expectancy')
high_corr = corrs[corrs.abs() > 0.5]
print("Variables have correlation larger than 0.5: ")
print(high_corr)

Corralations between with and without outliers does not change too much, which indicates
that the outliers does not generally affect our data
->Confirm assumption: OUTLIERS ARE NOT REMOVED

In [ ]:
# update data use for regress
df_regress = df_reg_y[["Life expectancy", "BMI", "Adult Mortality"]]

## Least square regression

In [ ]:
X1 = df_regress['Adult Mortality']
X2 = df_regress['BMI']
y = df_regress['Life expectancy']

model1 = smf.ols(formula='y ~ X1', data=df_regress).fit()
print(model1.summary())

In [ ]:
model2 = smf.ols(formula='y ~ X2', data=df_regress).fit()
print(model2.summary())

In [ ]:
model3 = smf.ols(formula='y ~ X1 + X2', data=df_regress).fit()
print(model3.summary())

## Check residuals for 4 assumptions

### Assumption 1: Linearity

In [ ]:
models = [model1, model2, model3]
plt.figure(figsize=(6, 10))
for i, model in enumerate(models):
    plt.subplot(3, 1, i+1)
    sns.residplot(x=model.fittedvalues, y=model.resid, lowess=True, line_kws={'color': 'red', 'lw': 1})
    plt.xlabel('Fitted values')
    plt.ylabel('Residuals')
    plt.title(f'Model {i+1}')
plt.tight_layout()
plt.show()

### Assumption 2: Independent

In [ ]:
for i, model in enumerate(models):
    d = durbin_watson(model.resid)
    print(f"Model {i+1}: {d}")

    if d<1.5: print("The residuals are positive autocorrelated.\n")
    elif d>2.5: print("The residuals are negative autocorrelated.\n")
    else: print("There is no correlation among the residuals.\n")

H0 (null hypothesis): There is no correlation among the residuals.
HA (alternative hypothesis): The residuals are autocorrelated.

The test statistic always ranges from 0 to 4 where:
- d = 2 indicates no autocorrelation
- d < 2 indicates positive serial correlation
- d > 2 indicates negative serial correlation

In general, if d is less than 1.5 or greater than 2.5 then there is potentially a serious autocorrelation problem.
Otherwise, if d is between 1.5 and 2.5 then autocorrelation is likely not a cause for concern.

### Assumption 3: Homoscedasticity

#### Determine by Graphic

In [ ]:
plt.figure(figsize=(6, 10))
for i, model in enumerate(models):
    plt.subplot(3, 1, i+1)
    sns.residplot(x=model.fittedvalues, y=model.resid, lowess=True, line_kws={'color': 'red', 'lw': 1})
    plt.xlabel('Fitted values')
    plt.ylabel('Residuals')
    plt.title(f'Model {i+1}')
plt.tight_layout()
plt.show()

Model 1:
- The residuals appear to fan out more as the fitted values increase, especially beyond 70. This indicates an increase in variance of the residuals with higher fitted values, suggesting the presence of heteroscedasticity.
Model 2:
- The residuals show some clustering and appear to have a somewhat funnel-shaped pattern, with a wider spread at the lower and higher ends of the fitted values compared to the middle. This non-constant variance suggests the presence of heteroscedasticity.
Model 3:
- The residuals show a wider spread for lower fitted values and appear to narrow slightly for higher fitted values. This indicates non-constant variance of residuals across the range of fitted values, suggesting heteroscedasticity.

#### Determine by Test

In [ ]:
for i, model in enumerate(models):
    white_test = het_white(model.resid, model.model.exog)
    print(f"White test: test statistic = {white_test[0]}, p-value = {white_test[1]}")
    if white_test[1] < 0.05:
        print("Heteroscedasticity is present (residuals are not equally scattered)\n")
    else:
        print("Homoscedasticity is present (residuals are equally scattered)\n")

### Assumption 4: Normality of Residuals

In [ ]:
sm.qqplot(model1.resid, line='45');
sm.qqplot(model2.resid, line='45');
sm.qqplot(model3.resid, line='45');

## Normality check

In [ ]:
# add semicolon to prevent duplicated graph issue
sm.qqplot(df_regress['Adult Mortality'], line='45');
sm.qqplot(df_regress['BMI'], line='45');
sm.qqplot(df_regress["Life expectancy"], line='45');

In [ ]:
# use Shapiro-Wilk test to test again
for col in df_regress.columns:
    test = shapiro(df_regress[col])
    print(f'Test statistic = {test[0]}, p-value = {test[1]}')
    if test[1] > 0.05: print(f'{col} looks normal distributed (fail to reject H0)\n')
    else: print(f'{col} does not normal distributed (reject H0)\n')

Remark:
Both the Q-Q plot and the Shapiro-Wilk test indicate that our dataset is not normally distributed. However, as our dataset is a sample from the years 2000 to 2015 and  continues to be updated annually, the increase in size may lead it to approximate normality over time, following the Central Limit Theorem (CLT). The CLT suggests that  with a large enough sample size, the sampling distribution of the mean will approximate a normal distribution, regardless of the initial distribution of the data. This principle is particularly important in inferential statistics, where the assumption of normality supports the validity of various statistical tests and the calculation of confidence intervals.

## Construct confidence interval


### Mean

In [ ]:
import numpy as np
from scipy.stats import t

n_bootstrap = 1000
def CI_mean(data, name):
# a) Confidence interval for the mean using bootstrapping
  data_means = []
  for _ in range(n_bootstrap):
    data_boot = np.random.choice(data, size=len(data), replace=True)
    data_means.append(np.mean(data_boot))
  data_ci_low = np.percentile(data_means, 2.5)
  data_ci_high = np.percentile(data_means, 97.5)
  print(f"Confidence interval for mean {name}: ({data_ci_low:.2f}, {data_ci_high:.2f})")
CI_mean(df_regress['BMI'], "BMI")
CI_mean(df_regress['Adult Mortality'], 'Adult Mortality')
CI_mean(df_regress['Life expectancy'], 'Life expectancy')

### Variance

In [ ]:
def CI_var(data, name):
  data_vars = []
  for _ in range(n_bootstrap):
    data_boot = np.random.choice(data, size=len(data), replace=True)
    data_vars.append(np.var(data_boot))  
  data_var_ci_low = np.percentile(data_vars, 2.5)
  data_var_ci_high = np.percentile(data_vars, 97.5)

  print(f"Confidence interval for variance {name}: ({data_var_ci_low:.2f}, {data_var_ci_high:.2f})")
CI_var(df_regress['BMI'], "BMI")
CI_var(df_regress['Adult Mortality'], 'Adult Mortality')
CI_var(df_regress['Life expectancy'], 'Life expectancy')

### Proportion

In [ ]:
#Confidence interval for proportion of Life expectancy higher than 60
import numpy as np
from scipy.stats import binom

# Assuming you have the dataframe 'df' with columns 'EPDS' and 'PROMIS_Anxiety'

# Define the significance level
alpha = 0.05

# a) Confidence interval for the proportion of EPDS >= 13
LElarger60 = df_regress[df_regress['Life expectancy'] >= 60]
LE_total = len(df_regress['Life expectancy'])
LE_larger60_len = len(LElarger60)
LE_larger60_proportion = LE_larger60_len / LE_total


LElarger60_ci_low = binom.ppf(alpha/2, LE_total, LE_larger60_proportion) / LE_total
LElarger60_ci_high = binom.ppf(1 - alpha/2, LE_total, LE_larger60_proportion) / LE_total

print(f"Confidence interval for proportion of Life Expectancy larger than 60 : ({LElarger60_ci_low:.4f}, {LElarger60_ci_high:.4f})")

## Perform hypothesis testing

### Propotion

In [ ]:
def ztest_propotion(n_counts, n_trials, p0, alpha, method, tail_type):
    p_hat = n_counts/n_trials
    z_test = (p_hat - p0) / np.sqrt(p0 * (1 - p0) / n_trials)

    match(method):
        case "c":
            match(tail_type):
                case "left":
                    z_crit = stats.norm.ppf(1-alpha)
                    if z_test < z_crit: 
                        return "Reject null hypothesis."
                    else: 
                        return "Fail to reject null hypothesis."
                case "right":
                    z_crit = stats.norm.ppf(alpha)
                    if z_test > z_crit:
                        return "Reject null hypothesis."
                    else: 
                        return "Fail to reject null hypothesis."
                case "two":
                    # 1-alpha/2 or alpha/2? i think both are ok
                    # since 1-alpha/2 = -alpha/2
                    z_crit = stats.norm.ppf(alpha/2)
                    # z_crit = stats.norm.ppf(1-alpha/2)
                    if abs(z_test) > z_crit:
                        return "Reject null hypothesis."
                    else: 
                        return "Fail to reject null hypothesis."
                case _:
                    return "Invalid option."
        case "p":
            match(tail_type):
                case "left":
                    p_val = stats.norm.cdf(z_test)
                case "right":
                    p_val = 1 - stats.norm.cdf(z_test)
                case "two":
                    p_val = 2 * (1- stats.norm.cdf(z_test))
                case _:
                    return "Invalid option."
            if (p_val < alpha):
                return "Reject null hypothesis."
            else: 
                return "Fail to reject null hypothesis."

# test if the propotion of Life expectancy < 50 is actually in the CI calculate above
counts = len(df_regress[(df_regress["Life expectancy"]<50)])
trials = len(df_regress)
alpha = 0.05
p0 = 0.8
print("Hypothesis testing using critical value: ")
print(ztest_propotion(counts, trials, p0, alpha, "c", "right"))
print()
print("Hypothesis testing using p value: ")
print(ztest_propotion(counts, trials, p0, alpha, "p", "right"))

### Mean

In [ ]:
# test if the mean of Life expectancy < 50 is actually in the CI calculate above
def ttest_mean(trials_mean, trials_std, n_trials, u0, alpha, method,tail_type):
    t_test = (trials_mean - u0) / (trials_std * np.sqrt(n_trials))

    match(method):
        case "c":
            match(tail_type):
                case "left":
                    t_crit = stats.t.ppf(q = alpha, df=n_trials-1)
                    if t_test < t_crit:
                        return "Reject null hypothesis."
                    else:
                        return "Fail to reject null hypothesis."
                case "right":
                    t_crit = stats.t.ppf(q = 1 - alpha, df=n_trials-1)
                    if t_test > t_crit:
                        return "Reject null hypothesis."
                    else:
                        return "Fail to reject null hypothesis."
                case "two":
                    # alpha/2 or 1-alpha/2?: both are ok
                    t_crit = stats.t.ppf(q = 1 - alpha / 2, df=n_trials-1)
                    # t_crit = stats.t.ppf(q = alpha / 2, df=n_trials-1)
                    if abs(t_test) > abs(t_crit):
                        return "Reject null hypothesis."
                    else:
                        return "Fail to reject null hypothesis."
                case _:
                    return "Invalid option."
        case "p":
            match(tail_type):
                case "left":
                    p_val = stats.t.cdf(t_test, df=n_trials-1)
                case "right":
                    p_val = 1 - stats.t.cdf(t_test, df=n_trials-1)
                case "two":
                    p_val = 2 * (1 - stats.t.cdf(abs(t_test), df=n_trials-1))
                case _:
                    return "Invalid option."
            if (p_val < alpha):
                return "Reject null hypothesis."
            else: 
                return "Fail to reject null hypothesis."
        case _:
            return "Invalid option."

sample_mean = np.mean(df_regress["BMI"])
sample_std = np.std(df_regress["BMI"], ddof=1)
sample_size = len(df_regress)
mean_test = 10
alpha = 0.05

print("Hypothesis testing using critical value: ")
print(ttest_mean(sample_mean, sample_std, sample_size, mean_test, alpha, "c", "right"))
print()
print("Hypothesis testing using p value: ")
print(ttest_mean(sample_mean, sample_std, sample_size, mean_test, alpha, "p", "right"))

### Standard Deviation

In [ ]:
def chitest_std(trials_std, n_trials, std0, alpha, method, tail_type):
    X2_test = (n_trials-1) * trials_std**2 / std0**2

    match(method):
        case "c":
            match(tail_type):
                case "left":
                    X2_crit = stats.chi2.ppf(q = 1 - alpha, df= n_trials-1)
                    if X2_test < X2_crit:
                        return "Reject null hypothesis."
                    else:
                        return "Fail to reject null hypothesis."
                case "right":
                    X2_crit = stats.chi2.ppf(q = alpha, df= n_trials-1)
                    if X2_test > X2_crit:
                        return "Reject null hypothesis."
                    else:
                        return "Fail to reject null hypothesis."
                case "two":
                    X2_crit_low = stats.chi2.ppf(q= alpha / 2, df=n_trials-1)
                    X2_crit_high = stats.chi2.ppf(q= 1 - alpha / 2, df=n_trials-1)
                    if (X2_test < X2_crit_low) | (X2_test > X2_crit_high):
                        return "Reject null hypothesis."
                    else:
                        return "Fail to reject null hypothesis."
                case _:
                    return "Invalid option."
        case "p":
            match(tail_type):
                case "left":
                    p_val = stats.chi2.cdf(X2_test, df=n_trials-1)
                case "right":
                    p_val = 1 - stats.chi2.cdf(X2_test, df=n_trials-1)
                case "two":
                    p_val = 2 * min(stats.chi2.cdf(X2_test, df= n_trials - 1),
                                              1 - stats.chi2.cdf(X2_test, df= n_trials - 1))
                case _:
                    return "Invalid option."
            if (p_val < alpha):
                return "Reject null hypothesis."
            else: 
                return "Fail to reject null hypothesis."
        case _:
            return "Invalid option."

sample_std = np.std(df_regress["BMI"], ddof=1)
sample_size = len(df_regress)
std_test = 21
alpha = 0.05
print("Hypothesis testing using critical value: ")
print(chitest_std(sample_std, sample_size, std_test, alpha, "c", "left"))
print()
print("Hypothesis testing using p value: ")
print(chitest_std(sample_std, sample_size, std_test, alpha, "p", "left"))